## dotenv and chatmodels setup

In [3]:
from dotenv import load_dotenv, find_dotenv
import os

load_dotenv(find_dotenv(), override=True)

True

In [5]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="llama3-8b-8192", temperature=0.1)

response = llm.invoke("What is the capital of Finland?")
print(response.content)

The capital of Finland is Helsinki.


In [6]:
from langchain.prompts import PromptTemplate
from langchain import hub
from langchain.agents import initialize_agent, Tool, AgentExecutor,create_react_agent
from langchain.tools import DuckDuckGoSearchRun, WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_experimental.tools.python.tool import PythonREPLTool

### Template setup

In [ ]:
template="""
Answer the question as truthfully as possible.
If you don't know the answer, say "I don't know".
Question: {question}
"""

prompt_template = PromptTemplate.from_template(template)

prompt = hub.pull("hwchase17/react")

#print(type(prompt))

#print(prompt.input_variables)

#print(prompt.template)

c:\Users\Shah\Desktop\langchain-playground\.venv\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


<class 'langchain_core.prompts.prompt.PromptTemplate'>
['agent_scratchpad', 'input', 'tool_names', 'tools']
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


### Tools Setup

In [21]:
# Python REPL Tool - for executing Python code
python_repl = PythonREPLTool()

python_repl_tool = Tool(
    name="Python REPL",
    func=python_repl.run,
    description="Useful for when you need to answer questions about Python code. Input should be a valid Python expression."
)

# Wikipedia Tool - for querying Wikipedia
api_wrapper = WikipediaAPIWrapper()
wikipedia = WikipediaQueryRun(api_wrapper=api_wrapper)

wikipedia_tool = Tool(
    name="Wikipedia",
    func=wikipedia.run,
    description="Useful for when you need to answer questions about Wikipedia articles. Input should be a valid Wikipedia query."
)

# DuckDuckGo Search Tool - for searching the web
search = DuckDuckGoSearchRun()
duckduckgo_tool = Tool(
    name="DuckDuckGo Search",
    func=search.run,
    description="Useful for when you need to answer questions about current events or general knowledge. Input should be a valid search query."
)

tools = [python_repl_tool, wikipedia_tool, duckduckgo_tool]

### Creating Agent

In [23]:
agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=prompt
)

agent_executor = AgentExecutor(
    agent=agent, 
    tools=tools, 
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=10)


### Test out the agent

In [28]:
question = "Generate first 10 numbers of the Fibonacci series"
response = agent_executor.invoke({'input': prompt_template.format(question=question)})
#print(response)



> Entering new AgentExecutor chain...
Thought: To generate the first 10 numbers of the Fibonacci series, I can use the Python REPL tool.

Action: Python REPL
Action Input: `fibonacci_sequence = [0, 1]; for i in range(8): fibonacci_sequence.append(fibonacci_sequence[-1] + fibonacci_sequence[-2]); print(fibonacci_sequence)`SyntaxError('invalid syntax', ('<string>', 1, 30, 'fibonacci_sequence = [0, 1]; for i in range(8): fibonacci_sequence.append(fibonacci_sequence[-1] + fibonacci_sequence[-2]); print(fibonacci_sequence)\n', 1, 33))Thought: It seems that the Python REPL tool is not correctly interpreting the input code. I'll try to fix the syntax error.

Action: Python REPL
Action Input: `fibonacci_sequence = [0, 1]; for i in range(8): fibonacci_sequence.append(fibonacci_sequence[-1] + fibonacci_sequence[-2]); print(fibonacci_sequence)`SyntaxError('invalid syntax', ('<string>', 1, 30, 'fibonacci_sequence = [0, 1]; for i in range(8): fibonacci_sequence.append(fibonacci_sequence[-1] + fib

In [29]:
question = "What is the capital of Finland?"
response = agent_executor.invoke({'input': prompt_template.format(question=question)})
print(response['output'])



> Entering new AgentExecutor chain...
Thought: This question seems to be asking about geography, specifically the capital of a country. I'm not sure what the best way to answer this question is, but I have access to tools that might be able to help.

Action: Wikipedia
Action Input: "capital of Finland"Page: Helsinki
Summary: Helsinki (Swedish: Helsingfors) is the capital and most populous city in Finland. It is on the shore of the Gulf of Finland and is the seat of southern Finland's Uusimaa region. About 687,000 people live in the municipality, with 1.3 million in the capital region and 1.6 million in the metropolitan area. As the most populous urban area in Finland, it is the country's most significant centre for politics, education, finance, culture, and research. Helsinki is 80 kilometres (50 mi) north of Tallinn, Estonia, 400 kilometres (250 mi) east of Stockholm, Sweden, and 300 kilometres (190 mi) west of Saint Petersburg, Russia. Helsinki has significant historical connection

In [32]:
question = "Who is current prime minister of UK?"
response = agent_executor.invoke({'input': prompt_template.format(question=question)})
print(response['output'])



> Entering new AgentExecutor chain...
Thought: I need to find the current prime minister of the UK. I can use Wikipedia to find the answer.

Action: Wikipedia
Action Input: "Current Prime Minister of the UK"Page: List of prime ministers of the United Kingdom
Summary: The prime minister of the United Kingdom is the principal minister of the crown of His Majesty's Government, and the head of the British Cabinet. 
There is no specific date for when the office of prime minister first appeared, as the role was not created but rather evolved over time through a merger of duties. The term was regularly, if informally, used by Robert Walpole by the 1730s. It was used in the House of Commons as early as 1805, and it was certainly in parliamentary use by the 1880s, although did not become the official title until 1905, when Arthur Balfour was prime minister.  
Historians generally consider Robert Walpole, who led the government of the Kingdom of Great Britain for over twenty years from 1721, t